Create a geoJSON file storing the number of partipants per country.

In [1]:
import os
import glob
import logging
from collections import Counter
import geojson
import json
import liegecolloquium

Prepare logger:

In [2]:
logger = logging.getLogger('counting')
logger.setLevel(logging.DEBUG)

# Files and directories

We need:
1. a geoJSON file with the world countries
2. a list of CSV files storing with the list of participants.

In [3]:
datadir = "../data/"
countryfile = os.path.join(datadir, "countries.geo.json")
participantfilelist = sorted(glob.glob(os.path.join(datadir, "ParticipantList-19*.tsv")))
outputdir = "../data/"
outputfile = os.path.join(outputdir, "ParticipantCountry.geojson")
logger.info("Working on {0} participant files".format(len(participantfilelist)))

# Read data
We provide a list of files to be read in order to create a list of participants for the period of interest (full period, decades, ...)

## Participant list

In [4]:
lonlist, latlist, countrylist = [], [], []
nptotal = 0
for participantfile in participantfilelist:
    logger.debug("Working on file {0}".format(os.path.basename(participantfile)))
    with open(participantfile, "r") as f:
        for line in f:
            l = line.rstrip().split('\t')
            participant = liegecolloquium.Participant(l[0], l[1], l[2], l[3], l[4])
            participant.replace_country()
            #lonlist.append(l[5])
            #latlist.append(l[4])
            countrylist.append(participant.country)
    logging.info("Cumulated number of participants: {0}".format(len(countrylist)))

DEBUG:counting:Working on file ParticipantList-1973.tsv
DEBUG:counting:Working on file ParticipantList-1974.tsv
DEBUG:counting:Working on file ParticipantList-1975.tsv
DEBUG:counting:Working on file ParticipantList-1976.tsv
DEBUG:counting:Working on file ParticipantList-1977.tsv
DEBUG:counting:Working on file ParticipantList-1978.tsv
DEBUG:counting:Working on file ParticipantList-1979.tsv
DEBUG:counting:Working on file ParticipantList-1980.tsv
DEBUG:counting:Working on file ParticipantList-1981.tsv
DEBUG:counting:Working on file ParticipantList-1982.tsv


## Count participant per country
Use the `Counter` function and then sort.

In [5]:
ParticipantPerCountry = Counter(countrylist)
SortedParticipant = sorted(ParticipantPerCountry.keys())

# Add *participant* property to the country JSON file 

The geoJSON files contains the boundaries of the countries.    
We only have to add a new property to each feature, the number of participants, so that they can be read by Leaflet.

First we read the geoJSON file, which returns a dictionary.    
Then we will add the property *participants* for the countries that are in our list.

In [6]:
with open(countryfile, 'r') as f:
    CountryJson = json.load(f)
type(CountryJson)

dict

In [7]:
# Loop on the countries 
nc = 0
np = 0
for countries in CountryJson['features']:
    # Extract the country name
    countryName = countries['properties']['name']
    # If country name in the initial list, write number of participants
    if countryName in countrylist:
        nc += 1
        logger.info("{0} --> {1}".format(countryName, ParticipantPerCountry[countryName]))
        countries['properties']['Participants'] = ParticipantPerCountry[countryName]
        np += ParticipantPerCountry[countryName]
        
logger.info("{0} countries and {1} participants (should be {2}) ".format(nc, np, 
                                                                         len(countrylist)))

INFO:counting:Australia --> 3
INFO:counting:Burundi --> 2
INFO:counting:Belgium --> 186
INFO:counting:Bulgaria --> 1
INFO:counting:Brazil --> 1
INFO:counting:Canada --> 26
INFO:counting:Switzerland --> 1
INFO:counting:Ivory Coast --> 1
INFO:counting:Germany --> 41
INFO:counting:Denmark --> 9
INFO:counting:Algeria --> 2
INFO:counting:Spain --> 5
INFO:counting:Estonia --> 15
INFO:counting:Finland --> 2
INFO:counting:France --> 89
INFO:counting:United Kingdom --> 53
INFO:counting:Guinea --> 4
INFO:counting:Greece --> 2
INFO:counting:Ireland --> 3
INFO:counting:Italy --> 14
INFO:counting:Mexico --> 5
INFO:counting:Niger --> 3
INFO:counting:Netherlands --> 32
INFO:counting:Norway --> 8
INFO:counting:Peru --> 1
INFO:counting:Russia --> 4
INFO:counting:Senegal --> 1
INFO:counting:Republic of Serbia --> 1
INFO:counting:Sweden --> 6
INFO:counting:Thailand --> 1
INFO:counting:United States of America --> 106
INFO:counting:31 countries and 628 participants (should be 628) 


## Create a new geoJSON with the additional properties
Now we can write the new geoJSON that `Leaflet` will ingest to produce a nice choropleth map.

In [9]:
with open(outputfile, 'w') as fp:
    fp.write("var participants = ")
    json.dump(CountryJson, fp)
logging.info("Output file created in directory {0}".format(outputdir))

In [10]:
# Loop on the countries 
for countries in CountryJson['features']:
    # Extract the country name
    print(countries['properties']['name'])

Afghanistan
Angola
Albania
United Arab Emirates
Argentina
Armenia
Antarctica
French Southern and Antarctic Lands
Australia
Austria
Azerbaijan
Burundi
Belgium
Benin
Burkina Faso
Bangladesh
Bulgaria
The Bahamas
Bosnia and Herzegovina
Belarus
Belize
Bermuda
Bolivia
Brazil
Brunei
Bhutan
Botswana
Central African Republic
Canada
Switzerland
Chile
China
Ivory Coast
Cameroon
Democratic Republic of the Congo
Republic of the Congo
Colombia
Costa Rica
Cuba
Northern Cyprus
Cyprus
Czech Republic
Germany
Djibouti
Denmark
Dominican Republic
Algeria
Ecuador
Egypt
Eritrea
Spain
Estonia
Ethiopia
Finland
Fiji
Falkland Islands
France
Gabon
United Kingdom
Georgia
Ghana
Guinea
Gambia
Guinea Bissau
Equatorial Guinea
Greece
Greenland
Guatemala
French Guiana
Guyana
Honduras
Croatia
Haiti
Hungary
Indonesia
India
Ireland
Iran
Iraq
Iceland
Israel
Italy
Jamaica
Jordan
Japan
Kazakhstan
Kenya
Kyrgyzstan
Cambodia
South Korea
Kosovo
Kuwait
Laos
Lebanon
Liberia
Libya
Sri Lanka
Lesotho
Lithuania
Luxembourg
Latvia
Morocc

In [12]:
ParticipantPerCountry

Counter({'Algeria': 2,
         'Australia': 3,
         'Belgium': 186,
         'Brazil': 1,
         'Bulgaria': 1,
         'Burundi': 2,
         'Canada': 26,
         'Denmark': 9,
         'Estonia': 15,
         'Finland': 2,
         'France': 89,
         'Germany': 41,
         'Greece': 2,
         'Guinea': 4,
         'Ireland': 3,
         'Italy': 14,
         'Ivory Coast': 1,
         'Mexico': 5,
         'Netherlands': 32,
         'Niger': 3,
         'Norway': 8,
         'Peru': 1,
         'Republic of Serbia': 1,
         'Russia': 4,
         'Senegal': 1,
         'Spain': 5,
         'Sweden': 6,
         'Switzerland': 1,
         'Thailand': 1,
         'United Kingdom': 53,
         'United States of America': 106})